# Shark attack log

In [1]:
# Libraries
library(tidyverse)
library(readxl)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
# Helpers
quiet <- function(x) {
    # Suppress both messages and warnings
    suppressMessages(suppressWarnings(x))
}

clean_str <- function(s) {
    # Convert to lower case and trim whitespaces
    return(str_to_lower(str_squish(s)))
}

In [3]:
# Data
file_url <- r"(https://www.sharkattackfile.net/spreadsheets/GSAF5.xls)"

    # Load up the spreadsheet into a temporary file 
temp_file <- tempfile(fileext = ".xls")
download.file(file_url, temp_file, mode = "wb")

In [4]:
df <- quiet(
        read_excel(
            temp_file,
            col_names = TRUE, 
            col_types = NULL
        )
    )
# df |> head(n=10)

In [5]:
# df structure
df |> str()

tibble [7,058 × 23] (S3: tbl_df/tbl/data.frame)
 $ Date            : chr [1:7058] "27th November" "27th November" "10th November" "9th November" ...
 $ Year            : chr [1:7058] "2025" "2025" "2025" "2025" ...
 $ Type            : chr [1:7058] "Unprovoked" "Unprovoked" "Unprovoked" "Unprovoked" ...
 $ Country         : chr [1:7058] "Australia" "Australia" "Australia" "French Polynesia" ...
 $ State           : chr [1:7058] "NSW" "NSW" "Western Australia" "Marquesas Islands" ...
 $ Location        : chr [1:7058] "Crowdy Bay" "Crowdy Bay" "Prevelly Beach Magaret River" "Hakahau Bay" ...
 $ Activity        : chr [1:7058] "Swimming" "Swimming" "Foil Boarding" "Swimming" ...
 $ Name            : chr [1:7058] "Lukas Schindler" "Livia Mulheim" "Andy McDonald" "Not stated (Dentist)" ...
 $ Sex             : chr [1:7058] "M" "F" "M" "M" ...
 $ Age             : chr [1:7058] "26" "25" "61" "40" ...
 $ Injury          : chr [1:7058] "Serious leg injuries" "Not stated" "No Injury to self" "De

In [6]:
# df dimensions
df |> dim()

[1] 7058   23

## Clean up

In [7]:
# Clean up the data frame
df <- df |>
    janitor::clean_names() |>
    mutate(
        # Year as integer
        year = as.integer(year),
        # Type of attack as factor
        type = factor(
            clean_str(type), 
            levels = c("unprovoked", "provoked", "questionable", "watercraft")
        ),
        # Age as integer
        age = as.integer(age),
        # Sex as factor
        sex = factor(
            clean_str(sex), 
            levels = c("m", "f")
        ),
        # Binary fatality status
        fatal = factor(
                case_match(
                    fatal_y_n,
                    "Y" ~ 1,
                    "N" ~ 0,
                    .default = NA_integer_
                ),
            levels = c(0, 1)
        )
    ) |>
    mutate(
        across(
            # Convert to lower case and purge whitespaces
            .cols = c("date", "time", "activity", "injury", "species"),
            .fns = ~ str_to_lower(str_squish(.x))
        )
    ) |>
    mutate(
        across(
            # Purge whitespaces
            .cols = c("country", "state", "location", "name"),
            .fns = ~ str_squish(.x)
        )
    ) |>
    select(-fatal_y_n)

Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `age = as.integer(age)`.
Caused by warning:
! NAs introduced by coercion"


###  Clean the time column

- recode `time` as time
- construct a `time_of_day` column

In [8]:
# Extract the raw column
raw_time <- df$time

# Clean up the values
hrs_mins <- str_match(raw_time, "(\\d{1,2})[a-z]*?(\\d{2})")
clean_time <- ifelse(!is.na(hrs_mins)[,1], paste(hrs_mins[,2], ":", hrs_mins[,3]), NA)

# Override the original column with clean time values
df["time"] <- hm(clean_time)

# Classify phases of day
day_phases <- r"(daybreak: 04:00–06:59
morning: 07:00–11:59
afternoon: 12:00–16:59
nightfall: 17:00–20:59
night: 21:00–03:59)"

# Match the string components
day_phases_components <- str_match_all(day_phases, "(\\w+):\\s*(\\d{2}:\\d{2})[–-](\\d{2}:\\d{2})")[[1]]

# Construct a key
day_phases_key <- tibble(
    phase = day_phases_components[,2],
    start = hm(day_phases_components[,3]),
    end = hm(day_phases_components[,4])
)

In [9]:
# Recode day phases
day_phases <- sapply(
    seq(nrow(df)),
    \(i) {
        t <- df$time[i]   # Or df[["time"]][i]
        raw_t <- raw_time[[i]]
        raw_s <- str_extract(raw_t, "[a-z ]+")

        if (!is.na(t)) {
            s_mask <- day_phases_key$start >= t
            e_mask <- day_phases_key$end <= t
            idx <- which(s_mask & e_mask)
            
            if (length(idx) == 1L) {
                phase <- day_phases_key[idx, "phase"]
                return(phase)
            } else {
                return("night")
            }
        } else if (!is.na(raw_s)) {
            a_match <- setdiff(
                unlist(str_split(raw_s, " ")),
                day_phases_key$phase
            )
            if (length(a_match) > 0) {
                return(raw_s)
            } else {
                return(NA_character_)
            }
        } else {
            return(NA_character_)
        }
    }
)

df <- df |>
    relocate(
        time,
        .after = "year"
    ) |>
    mutate(
        time_of_day = factor(day_phases, levels = unique(unlist(day_phases)))
    ) |>
    relocate(
        time_of_day,
        .after = "time"
    )

### Clean the date column

- recode `date` as date
- construct `date_notes` column

In [10]:
# Construct date_notes
df <- df |>
    mutate(
        date_notes = date
    )

In [11]:
# Construct a helper
clean_historical_date <- function(date_str, year_col = NA_integer_) {
      d <- date_str |>
        # Take reported for actual
        str_remove("^reported\\s+") |>
        # Remove suffixes
        str_replace_all("(?<=\\d)(st|nd|rd|th)", "")
  
    # Extract the year from the separate column
    year_present <- str_detect(d, "\\b\\d{4}\\b")
  
    d <- ifelse(
        !year_present & !is.na(year_col),
        paste(d, year_col),
        d
    )
  
    # Let lubridate parse the muddle
    parse_date_time(
        d,
        orders = c("d B Y", "B Y", "d-b-Y", "b-d-Y", "d B"),
        quiet = TRUE
    ) |> as.Date()
}

# Clean the column
df <- df |>
    mutate(
        date = clean_historical_date(date, year)
    )

## Clean species

- normalise `species` column
- construct `specimen_size` column

In [12]:
# Construct size columns
df <- df |>
    mutate(
        size_m = str_extract(species, "\\d*\\.?\\d+(?=m)"),
        size_ft = str_extract(species, "\\d*\\.?\\d+(?=ft)")
    )

In [13]:
# Clean the species column
df <- df |>
    mutate(
        species = str_replace_all(
            species,
            c("wfite" = "white",
              "shart" = "shark",
              "broze" = "bronze",
              "carribean" = "caribbean",
              "galapogas" = "galapagos",
              "shall" = "small")
        ),
        species =  case_when(
                str_detect(species, "oceanic whitetip") ~ "oceanic whitetip shark",
                str_detect(species, "blacktip reef") ~ "blacktip reef shark",
                str_detect(species, "caribbean reef") ~ "caribbean reef shark",
                str_detect(species, "sand tiger|raggedtooth")~ "sand tiger shark",
                str_detect(species, "sandbar") ~ "sandbar shark",
                str_detect(species, "bronze whaler") ~ "bronze whaler",
                str_detect(species, "wobbegong") ~ "wobbegong shark",
                str_detect(species, "sevengill") ~ "sevengill shark",
                str_detect(species, "cookiecutter") ~ "cookiecutter shark",
                str_detect(species, "reef shark") ~ "reef shark",
                 
                str_detect(species, "lemon") ~ "lemon shark",
                str_detect(species, "nurse") ~ "nurse shark",
                str_detect(species, "blacktip") ~ "blacktip shark",
                str_detect(species, "mako") ~ "mako shark",
                str_detect(species, "blue") ~ "blue shark",
                str_detect(species, "dusky") ~ "dusky shark",
                str_detect(species, "galapagos") ~ "galapagos shark",
                str_detect(species, "reef") ~ "reef shark",
                 
                str_detect(species, "bull") ~ "bull shark",
                str_detect(species, "tiger") ~ "tiger shark",
                str_detect(species, "great white|white shark|white shark") ~ "white shark",
                TRUE ~ NA_character_
            )


    )

## Final touches

- select cols
- filter out NA years

In [22]:
clean_df <- df |>
    select(date, year, time, time_of_day, type, country, state, location, activity, name, sex, injury, fatal, species, size_m, size_ft) |>
    filter(!is.na(year)) |>
    filter(year > 1900)

In [23]:
clean_df

date,year,time,time_of_day,type,country,state,location,activity,name,sex,injury,fatal,species,size_m,size_ft
<date>,<int>,<Period>,<fct>,<fct>,<chr>,<chr>,<chr>,<chr>,<chr>,<fct>,<chr>,<fct>,<chr>,<chr>,<chr>
2025-11-27,2025,6H 30M 0S,night,unprovoked,Australia,NSW,Crowdy Bay,swimming,Lukas Schindler,m,serious leg injuries,0,bull shark,3,NA
2025-11-27,2025,6H 30M 0S,night,unprovoked,Australia,NSW,Crowdy Bay,swimming,Livia Mulheim,f,not stated,1,bull shark,3,NA
2025-11-10,2025,17H 45M 0S,night,unprovoked,Australia,Western Australia,Prevelly Beach Magaret River,foil boarding,Andy McDonald,m,no injury to self,0,white shark,NA,NA
2025-11-09,2025,NA,not stated,unprovoked,French Polynesia,Marquesas Islands,Hakahau Bay,swimming,Not stated (Dentist),m,deep gash to bicep,0,NA,3,NA
2025-11-05,2025,NA,mid afternoon,unprovoked,USA,Hawaii,Pine Trees Hanalei Bay Kaui,swimming,Chance Swanson,m,injuries to legs,0,NA,NA,NA
2025-11-05,2025,NA,not stated,unprovoked,USA,Texas,Matagorda Beach Matagorda,fishing,Chuck Bledsoe,m,laceration on top and undermeath right foot,0,NA,NA,NA
2025-11-04,2025,18H 0M 0S,night,unprovoked,Samoa,NA,Aga Reef Resort Lalomanu,surfing,Evan Campbell,m,lacerations to right leg,0,tiger shark,NA,NA
2025-10-14,2025,NA,not stated,unprovoked,Columbia,"Bolivar, del Isolate",Catagena Province,swimming with sharks,Male child,m,severe hand injury,0,nurse shark,NA,NA
2025-10-11,2025,18H 23M 0S,night,unprovoked,Australia,Queensland,Cook Esplanade Thursday Island,fishing/swimming,Samuel Nai,m,serious abdonminal injuries,0,bull shark,NA,NA
